
## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
103
+-------------------+------------------+--------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|            coeff_1|           coeff_2|             coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+------------------+--------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-----------+---------+------+--------------------+------+
|-0.8959783423007046|1.0144895335220112|0.053078147428174224|    195.1|BSBSSSSB| 32.6875| -84.5197|       SNWD|0.9996906530843381|0.9992940522252314|0.9992929661407159|0.9977437409006402|USC0

In [14]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00098535,1993.0,-0.895978
1,USC00093271,1935.0,-0.895978
2,USC00095447,1935.0,-0.895978
3,USC00093271,1968.0,3.830884
4,USC00093936,1968.0,3.830884


In [15]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [16]:
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station),'fraction explained= ',1-MS(tbl_minus_station)/MS(year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),'fraction explained= ',1-MS(tbl_minus_year)/MS(year_station_table)

total MS                   =  9734.99735929
MS removing mean-by-station=  4443.87675531 fraction explained=  0.543515360991
MS removing mean-by-year   =  80.3140511889 fraction explained=  0.991749966823


In [12]:
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station
print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station),'fraction explained= ',1-MS(tbl_minus_station)/MS(year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),'fraction explained= ',1-MS(tbl_minus_year)/MS(year_station_table)

SELECT station, year, coeff_2 FROM weather
total MS                   =  5689.913501
MS removing mean-by-station=  4012.18198595 fraction explained=  0.29486063624
MS removing mean-by-year   =  1611.59505356 fraction explained=  0.71676282016


In [13]:
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station
print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station),'fraction explained= ',1-MS(tbl_minus_station)/MS(year_station_table)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year),'fraction explained= ',1-MS(tbl_minus_year)/MS(year_station_table)

SELECT station, year, coeff_3 FROM weather
total MS                   =  731.724942806
MS removing mean-by-station=  477.816011085 fraction explained=  0.34700051463
MS removing mean-by-year   =  618.131053812 fraction explained=  0.155241242096
